In [20]:
from copy import deepcopy
import torch.nn as nn
import torch
import types
from torch.autograd import Variable
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import pickle

In [7]:
import torch
import torch.nn as nn
from scene_net import *

TASKS_NUM_CLASS = [40, 3]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def count_parameters(model):
    # Function to count the number of parameters in each layer of a PyTorch model
    
    for name, param in model.named_parameters():
        print(f"Layer: {name} | Size: {param.size()} | Count: {param.numel()}")

    for name, param in model.named_buffers():
        print(f"Layer: {name} | Size: {param.size()} | Count: {param.numel()}")

print("made it here 1")

# Create an instance of the network
net = SceneNet(TASKS_NUM_CLASS).to(device)
#net.load_state_dict(torch.load("/home/nravi/DiSparse-Multitask-Model-Compression/results_new/best_nyuv2_baseline.pth"))

print("made it here 2")

# Count the parameters
#count_parameters(net)


made it here 1
made it here 2


In [5]:
def get_pruned_init(net):
    for module in net.modules():
        # Check if it's basic block
        if isinstance(module, nn.modules.conv.Conv2d) or isinstance(module, nn.modules.Linear):
            module = prune.identity(module, 'weight')
    return net

In [8]:
net = SceneNet(TASKS_NUM_CLASS).to(device)
net = get_pruned_init(net)
net.load_state_dict(torch.load("/home/nravi/DiSparse-Multitask-Model-Compression/results_new/nyuv2_disparse_pt_90.pth"))

# Count the parameters
count_parameters(net)


Layer: backbone.conv1.weight_orig | Size: torch.Size([64, 3, 7, 7]) | Count: 9408
Layer: backbone.bn1.weight | Size: torch.Size([64]) | Count: 64
Layer: backbone.bn1.bias | Size: torch.Size([64]) | Count: 64
Layer: backbone.blocks.0.0.conv1.weight_orig | Size: torch.Size([64, 64, 3, 3]) | Count: 36864
Layer: backbone.blocks.0.0.bn1.weight | Size: torch.Size([64]) | Count: 64
Layer: backbone.blocks.0.0.bn1.bias | Size: torch.Size([64]) | Count: 64
Layer: backbone.blocks.0.0.conv2.weight_orig | Size: torch.Size([64, 64, 3, 3]) | Count: 36864
Layer: backbone.blocks.0.0.bn2.weight | Size: torch.Size([64]) | Count: 64
Layer: backbone.blocks.0.0.bn2.bias | Size: torch.Size([64]) | Count: 64
Layer: backbone.blocks.0.1.conv1.weight_orig | Size: torch.Size([64, 64, 3, 3]) | Count: 36864
Layer: backbone.blocks.0.1.bn1.weight | Size: torch.Size([64]) | Count: 64
Layer: backbone.blocks.0.1.bn1.bias | Size: torch.Size([64]) | Count: 64
Layer: backbone.blocks.0.1.conv2.weight_orig | Size: torch.Size

In [16]:
print(torch.count_nonzero(net.backbone.blocks[3][2].conv2.weight_mask))

tensor(154623, device='cuda:0')


In [14]:
print((net.backbone.conv1.weight_mask).shape)

torch.Size([64, 3, 7, 7])


In [17]:
154623/2359296

0.0655377705891927

In [18]:
def layer_EPI(mask1, mask2):
    
    n1 = torch.count_nonzero(mask1).item()
    n2 = torch.count_nonzero(mask2).item()

    return abs(n1 - n2) / (n1 + n2)

In [55]:
def EPI(keep_mask1, keep_mask2, model):
    running_sum = 0
    count = 0

    for (name, module) in model.named_modules():
        print(name)
        if (isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear)) and ("backbone" in name):
            params = module.parameters()
            running_sum += layer_EPI(keep_mask1[count], keep_mask2[count])
            count += 1

    print(count)
    
    return 1 - running_sum/count


In [49]:
net = SceneNet(TASKS_NUM_CLASS).to(device)

In [60]:
with open("keepmasks.txt", "rb") as file:
    keepmasks = pickle.load(file)

epi = EPI(keep_mask1=keepmasks['seg'], keep_mask2=keepmasks['sn'], model=net)

len of keep mask 1:  48
len of keep mask 2:  48
len of mnp:  165

backbone
backbone.conv1
inside if
backbone.bn1
backbone.relu
backbone.maxpool
backbone.blocks
backbone.blocks.0
backbone.blocks.0.0
backbone.blocks.0.0.conv1
inside if
backbone.blocks.0.0.bn1
backbone.blocks.0.0.relu
backbone.blocks.0.0.conv2
inside if
backbone.blocks.0.0.bn2
backbone.blocks.0.1
backbone.blocks.0.1.conv1
inside if
backbone.blocks.0.1.bn1
backbone.blocks.0.1.relu
backbone.blocks.0.1.conv2
inside if
backbone.blocks.0.1.bn2
backbone.blocks.0.2
backbone.blocks.0.2.conv1
inside if
backbone.blocks.0.2.bn1
backbone.blocks.0.2.relu
backbone.blocks.0.2.conv2
inside if
backbone.blocks.0.2.bn2
backbone.blocks.1
backbone.blocks.1.0
backbone.blocks.1.0.conv1
inside if
backbone.blocks.1.0.bn1
backbone.blocks.1.0.relu
backbone.blocks.1.0.conv2
inside if
backbone.blocks.1.0.bn2
backbone.blocks.1.1
backbone.blocks.1.1.conv1
inside if
backbone.blocks.1.1.bn1
backbone.blocks.1.1.relu
backbone.blocks.1.1.conv2
inside if
bac

In [61]:
epi

0.7269788949033089

In [40]:
for name, layer in net.named_modules():
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
        print(name)

backbone.conv1
backbone.blocks.0.0.conv1
backbone.blocks.0.0.conv2
backbone.blocks.0.1.conv1
backbone.blocks.0.1.conv2
backbone.blocks.0.2.conv1
backbone.blocks.0.2.conv2
backbone.blocks.1.0.conv1
backbone.blocks.1.0.conv2
backbone.blocks.1.1.conv1
backbone.blocks.1.1.conv2
backbone.blocks.1.2.conv1
backbone.blocks.1.2.conv2
backbone.blocks.1.3.conv1
backbone.blocks.1.3.conv2
backbone.blocks.2.0.conv1
backbone.blocks.2.0.conv2
backbone.blocks.2.1.conv1
backbone.blocks.2.1.conv2
backbone.blocks.2.2.conv1
backbone.blocks.2.2.conv2
backbone.blocks.2.3.conv1
backbone.blocks.2.3.conv2
backbone.blocks.2.4.conv1
backbone.blocks.2.4.conv2
backbone.blocks.2.5.conv1
backbone.blocks.2.5.conv2
backbone.blocks.3.0.conv1
backbone.blocks.3.0.conv2
backbone.blocks.3.1.conv1
backbone.blocks.3.1.conv2
backbone.blocks.3.2.conv1
backbone.blocks.3.2.conv2
backbone.ds.1.0
backbone.ds.2.0
backbone.ds.3.0
task1_fc1_c0.conv1
task1_fc1_c0.conv2
task1_fc1_c0.conv3
task1_fc1_c1.conv1
task1_fc1_c1.conv2
task1_fc1_